In [2]:
from dataset import *
from processing import *
from models import *

In [8]:
pip install simpletransformers

     ---------------------------------------- 0.0/42.4 kB ? eta -:--:--
     --------- ------------------------------ 10.2/42.4 kB ? eta -:--:--
     --------- ------------------------------ 10.2/42.4 kB ? eta -:--:--
     ------------------ ------------------- 20.5/42.4 kB 131.3 kB/s eta 0:00:01
     ------------------------------------ - 41.0/42.4 kB 196.9 kB/s eta 0:00:01
     -------------------------------------- 42.4/42.4 kB 187.0 kB/s eta 0:00:00
     ---------------------------------------- 0.0/43.6 kB ? eta -:--:--
     ---------------------------------------- 43.6/43.6 kB 1.1 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata 


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\Draco\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
preprocess_midi_files('F:\\GitHub\\dataset\\midi_dataset', 'F:\\GitHub\\dataset\\np_dataset')

 [F:\GitHub\dataset\midi_dataset\Beethoven - Symphony no. 5.mid] [F:\GitHub\dataset\midi_dataset\Beethoven - Symphony no. 6 - 1st movement.mid] [F:\GitHub\dataset\midi_dataset\Beethoven - Symphony no. 7 - 2nd movement.mid] [F:\GitHub\dataset\midi_dataset\Beethoven - Symphony no. 9 - 2nd movement.mid] [F:\GitHub\dataset\midi_dataset\Beethoven - Symphony no. 9 - 4th movement.mid] [F:\GitHub\dataset\midi_dataset\Dukas - Sorcerer's Apprentice.mid] [F:\GitHub\dataset\midi_dataset\Hans_Zimmer_-_Pirates_Of_The_Caribbean_-_He's_A_Pirate.mid] [F:\GitHub\dataset\midi_dataset\Mozart - Eine Kleine Nachtmusik.mid] [F:\GitHub\dataset\midi_dataset\Mozart - Lacrimoza.mid] [F:\GitHub\dataset\midi_dataset\Mozart - Marige of Figaro.mid] [F:\GitHub\dataset\midi_dataset\Mozart - Queen of the Night.mid] [F:\GitHub\dataset\midi_dataset\Mozart - Symphony no. 40.mid] [F:\GitHub\dataset\midi_dataset\Mozart - Symphony no. 41 - 3rd movement.mid] [F:\GitHub\dataset\midi_dataset\Paul Dukas - Sorcerer's Apprentice.m

C:\Users\Draco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pretty_midi\pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


In [7]:
for batch in test_loader:
    print(batch[0].size())

torch.Size([1, 128])
torch.Size([1, 128])
